In [5]:
import cv2.aruco as aruco
import cv2
import numpy as np

dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters =  cv2.aruco.DetectorParameters()
# 
detector = aruco.ArucoDetector(dictionary, parameters)

In [2]:
board = aruco.CharucoBoard((8, 8), 0.4, 0.2, dictionary)

In [3]:
# show(board.generateImage((1024, 1024)))

In [111]:
img = cv2.imread("../machines/02_dobot/arucos.png")
if img.shape[-1] != 3:
    img_c = np.repeat(img[..., None], 3, axis=-1)
else:
    img_c = img.copy()

In [33]:
show(img)

In [10]:
images = [cv2.imread(f"../machines/02_dobot/charuco/charuco_{i}.png") for i in range(3)]
matched_points = []
corners_2 = []

for image in images:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cur_corners, cur_ids, _ = detector.detectMarkers(image)
    matched_points.append(list(
        map(np.squeeze, board.matchImagePoints(cur_corners, cur_ids))
    ))
    corners_2.append(aruco.interpolateCornersCharuco(
        cur_corners, cur_ids,
        img, board
    )[1:])
    
obj_points, img_points = zip(*matched_points)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    obj_points, img_points, image.shape,
    None, None
)

In [122]:
img_c = aruco.drawDetectedCornersCharuco(img_c, corners2, ids2, [255, 0, 0])
show(img_c)

In [14]:
for i in range(3):
    img_c = images[i].copy()

    t = np.zeros((2, 3))
    _, rvec, tvec = aruco.estimatePoseCharucoBoard(
        *corners_2[i],
        board, mtx, dist,
        None, None
    )

    img_c = cv2.drawFrameAxes(img_c, mtx, dist, rvec, tvec, 0.4)

    show(img_c)

In [12]:
def show(img):
    cv2.imshow("window", img)
    while True:
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    cv2.destroyAllWindows()